# Fluid modeling

PorePy supports the introduction of multiphase, multicomponent fluids into its models.

This tutorial focuses on relevant mixin-methods and two approaches to modeling of fluid properties:

- heuristic approach using functions in the form of constitutive laws
- Equation of State (EoS) -based approach using the respective object for computations

### Table of contents

1. [Basics and Default Behavior in PorePy](#basics-and-default-behavior-in-porepy)
2. [Modeling component and phase context](#modeling-component-and-phase-context)
3. Modeling of fluid properties
    - Heuristic approach (default)
    - Equation-of-state-approach
4. Overriding individual properties
5. EoS approach
    - The EquationOfState class
    - Defining dependencies of phase properties

## Basics and Default Behavior in PorePy

Fluids can be defined in terms of two contexts: components and phases.

While the number of components is something the modeler choses naturally, the number (and more importantly type) of phases is
an unknown number in the general thermodynamic sense.
In PorePy, the modeler must define the (maximal) number and types of phases expected in the course of simulations.

The modeling can then be roughly performed in 3 steps:

1. Defining the components in the fluid (water, CO2, ...)
2. Defining the expected phases (1 gas phase, 2 liquid phases for example)
3. Introducing a thermodynamic model for fluid properties (densities, enthalpies, ...)

> **Note on distinction between fluid, phase and component properties:**
> 
> It makes no sense to talk about the density of a component, say water.
> Liquid and gaseous water have greatly different densities.
> I.e., extensive properties like density and enthalpy are always related to the phase.
> Multiple components can be present in a phase and influence the properties.
> Extensive properties of fluid mixtures (consisting of multiple components in multiple phases), depend in a clearly defined manner on the respective properties of present phases (usually a weighted sum of phase properties).

The default fluid in PorePy models consists of 1 component, and 1 phase containing that component.
In this simple case, the fluid properties are equal to the (single) phase properties.

The class [`FluidMixin`](../src/porepy/compositional/compositional_mixins.py#L1042) provides respective functionalities for creating a fluid, and is part of all models provided in PorePy.

Legacy implementations provides a simple customization of the single-component without major effort:


In [8]:
import porepy as pp
from porepy.applications.material_values.fluid_values import water

model_params = {
    'material_constants': {
        'fluid': pp.FluidComponent(**water)
    }
}

model = pp.SinglePhaseFlow(model_params)
model.prepare_simulation()

Above code creates the default single-phase, single-component fluid and is ready for simulations right away.
It is compatible with all constitutive laws found in the [fluid property library](../src/porepy/models/fluid_property_library.py).

In [9]:
print(model.fluid.num_components)
print(model.fluid.num_phases)
print(model.fluid.reference_component.name)

1
1
water


Note however, that the `FluidMixin` is part of all models found in the `models` subpackage.
It comes with some general implementations of fluid properties, which are covered in a later section.
To override those general implementations, constitutive laws must be treated as pure mixins (as intended).
These constitutive laws then represent the thermodynamic model of the fluid.

For customization, they must be on top in the MRO:

In [ ]:
class CustomSinglePhaseFlow(
    pp.constitutive_laws.FluidDensityFromPressureAndTemperature,
    pp.SinglePhaseFlow,
):
    """Just for demonstration purpose. Note that the base model has no notion of
    temperature."""
    

## Modeling component and phase context

For customizing the component context, the method we need to override is `get_components`.
For customizing the phase context, we need to override `get_phase_configuration`.

Below example `MyFluid` will model a 2-component, 3-phase fluid, assuming that the simulation
is in pressure and temperature ranges where water and CO2 can separate into two immiscible, liquid-like phases.

In [11]:
from typing import Sequence


class MyFluid(pp.PorePyModel):

    def get_components(self) -> Sequence[pp.FluidComponent]:
        return [
            pp.FluidComponent(name='H2O'),
            pp.FluidComponent(name='CO2'),
        ]
    
    def get_phase_configuration(
        self, components: Sequence[pp.compositional.ComponentLike]
    ) -> Sequence[tuple[pp.compositional.PhysicalState, str]]:
        # Phases are configured by defining the physical state and a name.
        return [
            (pp.compositional.PhysicalState.liquid, 'aqueous'),
            (pp.compositional.PhysicalState.liquid, 'non_aqueous'),
            (pp.compositional.PhysicalState.gas, 'gaseous'),
        ]

Each component has an associated overall fraction variable accessible as `component.fraction`.
Each phase has an associated fraction variable *and* a saturation variable, accessible as
`phase.fraction` and `phase.saturation` respectively. The fractions can be either massic or molar and depend on the
implementation of the fluid properties. Saturations are always volumetric fractions.

By default, the first, returned component and phase (configuration) are set as the reference component and phase respectively.
This choice has an impact on both variables and equations created, depending on the model parametrization.

In [ ]:
model_params = {
    'eliminate_reference_phase': True,
    'eliminate_reference_component': True,
}

Both parameters are by default `True`. If the reference phase is eliminated, its fraction and saturation
are not introduced as independent variables, but expressed through unity:

$$
\varphi_{r} = 1 - \sum_{i\neq r} \varphi_i
$$

Analogously, if the reference component is eliminated, its overall fraction is expressed through unity.
Additionally, if the reference component is eliminated, compositional flow models do not introduce a mass balance equation
for the respective component.

We note these effects here but refer to the code in `porepy.models.compositional_flow` for more details.

Now that we know how to introduce phases and components into the model, we can talk about the interplay
between the two contexts: Components in a phase.

In general, a component in a phase is associated with an additional degree of freedom, the partial fraction `phase.partial_fraction_of[component]`

The behavior of which component is present in which phase is determined by the method `set_components_in_phases`.
Its default behavior introduces every component in every phase (unified assumption).
For fluids, this assumption is realistic since traces of every component can be found in every phase, if that phase is present in the thermodynamic sense (phase appearance/disappearance).
Future work will encompass solid components and solid phases, and expecting a fluid component like water inside a solid phase with granite or sand is nonsense. We can avoid introducing respective DoFs.

To describe that logic, let's assume a special model based on `MyFluid` which we simply such that each liquid phase contains only an associated component.

In [ ]:
class MySpecialFluid(MyFluid):

    def set_components_in_phases(
        self, components: Sequence[pp.Component], phases: Sequence[pp.Phase]
    ) -> None:
        h2o, co2 = components
        aqu, non_aqu, gas = phases

        # Aqueous phase contains only water, the non-aqueous only CO2.
        # The gas phase contains both, since gases always mix.
        aqu.components = [h2o]
        non_aqu.components = [co2]
        gas.components = [h2o, co2]

With above mixin `MySpecialFluid`, the two liquid phases will contain only 1 component.
This means that only 1 partial fraction per liquid phase can exist. If only 1 can exist, it is by default the scalar 1.
The gas phase has two partial fractions associated, one for each component. Since the unity constraint holds analogously for partial fractions, the partial fraction of water in the gas phase is eliminated **always**. This is contrary to the phase and overall fractions, where the choice of number of degrees of freedom is relevant to treat the issue of
phase appearance and disappearance.

To summarize:

With default model parameters, `MySpecialFluid` will introduce in total 6 degrees of freedom in each subdomain cell:

- 1 overall fraction of CO2
- 2 phase fractions for the non-aqueous and the gas phase
- 2 phase saturations for the non-aqueous and gas phase
- 1 partial fraction for CO2 in the gas phase.

This behavior, including some utility functions, is implemented in the [MixtureDOFHandler](../src/porepy/compositional/compositional_mixins.py#106), which is inherited by the
[CompositionalVariablesMixin](../src/porepy/compositional/compositional_mixins.py#536).
The latter must be part of any PorePy-model with more than 1 phase and component.

> **Note on phase fractions:**
>
> Phase fractions are quantities which do not usually appear in flow & transport problems, but in phase equilibrium or chemical problems.
> Hence it is not created as a DOF by default, but its creation must be triggered by defining an equilibrium type in the `model_params['equilibrium_type']= 'some_string'`.
>
> They can appear under non-isothermal conditions though, since the specific enthalpy and specific internal energy of the fluid mixture are sums of phase enthalpies/energies weighed with phase fractions.
> In this more complex models, the user must be well aware of the DOFs required and how to create them, and how the (specific) energies are implemented.